### Практическая работа к уроку №2

In [1]:
from math import log, exp

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    items = popular.item_id.values
    weights = popular.weights.values
    
    recs = np.random.choice(items, size=n, p=weights, replace=False)
        
    return recs.tolist()

In [6]:
#  Просуммируем стоимости покупок по каждому товару
popular = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popular.sort_values('sales_value', ascending=False, inplace=True)
# Отнормируем по сумме всех покупок, чтобы веса в сумме давали 1
sum_sale = popular.sales_value.sum()
popular['weights'] = popular.sales_value.apply(lambda x: x / sum_sale)

popular.head()

,item_id,sales_value,weights
55470,6534178,447799.94,0.063498
55430,6533889,40483.34,0.005741
28895,1029743,35764.66,0.005071
55465,6534166,30170.77,0.004278
34707,1082185,26029.96,0.003691


In [7]:
%%time

result['weighted_random'] = result['user_id'].apply(lambda x: weighted_random_recommendation(popular[['item_id', 'weights']]))

result.head()

CPU times: user 4.91 s, sys: 636 µs, total: 4.91 s
Wall time: 4.92 s


,user_id,actual,weighted_random
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[835630, 863762, 6533889, 8204756, 1084025]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[918635, 6534178, 1004906, 951412, 10344586]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[904129, 1113111, 936355, 5567319, 984680]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[8019902, 1037692, 865888, 893018, 13416246]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[6534178, 891649, 851468, 1072685, 8069350]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров

In [8]:
# топ-5000 товаров
n = 5000
popular = data_train.groupby('item_id')['sales_value'].sum().reset_index()
popular.sort_values('sales_value', ascending=False, inplace=True)
top5000 = popular.head(n).item_id
top5000.head()

55470    6534178
55430    6533889
28895    1029743
55465    6534166
34707    1082185
Name: item_id, dtype: int64

In [9]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [10]:
%%time

result['random_top_5000'] = result['user_id'].apply(lambda x: random_recommendation(top5000, n=5))

result.head(2)

CPU times: user 310 ms, sys: 111 µs, total: 310 ms
Wall time: 309 ms


,user_id,actual,weighted_random,random_top_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[835630, 863762, 6533889, 8204756, 1084025]","[7410335, 1000493, 6773219, 7168431, 1010950]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[918635, 6534178, 1004906, 951412, 10344586]","[819927, 8069118, 946995, 1135802, 871754]"


- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [11]:
# Будем проверять следующее количество соседей
neighbors = [1, 2, 3, 4, 5]

Создадим разряженную матрицу

In [12]:
# Отберем топ-5000 товаров и будем рекомендовать из них
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

print(f'Заполненность матрицы: {user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100} %')

# создаем словари мапинга между id бизнеса к строчному id матрицы
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

/home/mayo889/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Заполненность матрицы: 5.33770796861036 %


#### Item-Item Recommender

In [13]:
%%time

for neighbor in neighbors:
    
    model = ItemItemRecommender(K=neighbor, num_threads=0) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)
    
    new_column = f"itemitem_k_{neighbor}"
    
    result[new_column] = result['user_id'].\
        apply(lambda user_id: [id_to_itemid[rec[0]]
                               for rec in model.recommend(userid=userid_to_id[user_id],
                                                          user_items=sparse_user_item,   # на вход user-item matrix
                                                          N=5, 
                                                          filter_already_liked_items=False, 
                                                          filter_items=[itemid_to_id[6666]], 
                                                          recalculate_user=True)])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 15 s, sys: 297 ms, total: 15.3 s
Wall time: 5.5 s


#### Косинусное сходство и CosineRecommender

In [14]:
%%time

for neighbor in neighbors:
    
    model = CosineRecommender(K=neighbor, num_threads=0) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
              show_progress=True)
    
    new_column = f"cosine_k_{neighbor}"
    
    result[new_column] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]]
                         for rec in model.recommend(userid=userid_to_id[x], 
                                                    user_items=sparse_user_item,   # на вход user-item matrix
                                                    N=5, 
                                                    filter_already_liked_items=False, 
                                                    filter_items=[itemid_to_id[6666]], 
                                                    recalculate_user=True)])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 15.2 s, sys: 276 ms, total: 15.5 s
Wall time: 5.65 s


#### TF-IDF взвешивание и TFIDFRecommender

In [15]:
%%time

for neighbor in neighbors:
    
    model = TFIDFRecommender(K=neighbor, num_threads=4) # K - кол-во билжайших соседей

    model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
              show_progress=True)
    
    new_column = f"tfidf_k_{neighbor}"
    
    result[new_column] = result['user_id'].\
        apply(lambda x: [id_to_itemid[rec[0]] 
                         for rec in model.recommend(userid=userid_to_id[x], 
                                                    user_items=sparse_user_item,   # на вход user-item matrix
                                                    N=5, 
                                                    filter_already_liked_items=False, 
                                                    filter_items=[itemid_to_id[6666]], 
                                                    recalculate_user=False)])

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 15.2 s, sys: 211 ms, total: 15.4 s
Wall time: 5.59 s


#### Сравнение всех результатов

In [16]:
from metrics import precision_at_k

In [17]:
comparing = {}

for name_col in result.columns[1:]:
    comparing[name_col] = round(result.apply(lambda row: precision_at_k(row[name_col], row['actual'], k=5),
                                             axis=1).mean(), 
                                4)

/home/mayo889/Documents/GeekBrains/GeekBrains/recommendations/02_lesson/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


In [18]:
comparing = pd.DataFrame.from_dict(comparing, orient='index', columns=['precision@5'])
comparing.sort_values(by='precision@5', ascending=False, inplace=True)
comparing

,precision@5
actual,1.0000
itemitem_k_1,0.2199
itemitem_k_2,0.2193
itemitem_k_3,0.2190
tfidf_k_2,0.1795
cosine_k_2,0.1763
cosine_k_1,0.1730
tfidf_k_3,0.1688
itemitem_k_4,0.1673
tfidf_k_4,0.1644


Вывод: самое лучшее качество рекомендаций получено с помощью метода Item-Item Recommender при количестве соседей равным 1, 2 и 3